# Partition Switching

Uno de los beneficios del particionado es la capacidad de transferir los datos de una partición de una tabla hacia otra partición de forma casi instantanea, esto es sumamente útil cuando se trabaja con cargas de trabajo OLAP debido a que se puede cargar los datos en una tabla intermedia sin interferir en la tabla principal, reduciendo problemas de bloqueo y concurrencia, además no los reportes que involucren los nuevos datos se siempre se harán de forma completa una vez esten disponibles. Otra utilidad de partition switching es para el proceso de archiving, debido a que se pueden mover los datos de una forma muy rapida.

In [2]:
CREATE DATABASE PartitionTestDB

Commands completed successfully.

Total execution time: 00:00:00.510

In [3]:
USE PartitionTestDB


IF OBJECT_ID('dbo.GetNums') IS NOT NULL
  DROP FUNCTION dbo.GetNums;
GO
 
CREATE FUNCTION dbo.GetNums(@n AS BIGINT) RETURNS TABLE AS RETURN
  WITH
  L0   AS(SELECT 1 AS c UNION ALL SELECT 1),
  L1   AS(SELECT 1 AS c FROM L0 AS A CROSS JOIN L0 AS B),
  L2   AS(SELECT 1 AS c FROM L1 AS A CROSS JOIN L1 AS B),
  L3   AS(SELECT 1 AS c FROM L2 AS A CROSS JOIN L2 AS B),
  L4   AS(SELECT 1 AS c FROM L3 AS A CROSS JOIN L3 AS B),
  L5   AS(SELECT 1 AS c FROM L4 AS A CROSS JOIN L4 AS B),
  Nums AS(SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS n FROM L5)
  SELECT TOP (@n) n FROM Nums ORDER BY n;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

## Switch en tablas sin particionar

**1\. Setup de las tablas**

In [4]:
-- Drop objects if they already exist
IF EXISTS (SELECT * FROM sys.tables WHERE name = N'SalesSource')
  DROP TABLE SalesSource;
IF EXISTS (SELECT * FROM sys.tables WHERE name = N'SalesTarget')
  DROP TABLE SalesTarget;
-- Create the Non-Partitioned Source Table (Heap) on the [PRIMARY] filegroup
CREATE TABLE SalesSource (
  SalesDate DATE,
  Quantity INT
) ON [PRIMARY];

-- Insert test data
INSERT INTO SalesSource(SalesDate, Quantity)
SELECT DATEADD(DAY,dates.n-1,'2012-01-01') AS SalesDate, qty.n AS Quantity
FROM dbo.GetNums(DATEDIFF(DD,'2012-01-01','2016-01-01')) dates
CROSS JOIN dbo.GetNums(1000) AS qty;
-- Create the Non-Partitioned Target Table (Heap) on the [PRIMARY] filegroup
CREATE TABLE SalesTarget (
  SalesDate DATE,
  Quantity INT
) ON [PRIMARY];

(1461000 rows affected)

Total execution time: 00:00:05.821

**2\. Visualizar número de renglones en tabla objetivo y tabla destino**

In [5]:
SELECT COUNT(*) FROM SalesSource; 
SELECT COUNT(*) FROM SalesTarget; 

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.146

(No column name)
1461000


(No column name)
0


**3\. Realizar el intercambio de información**

In [6]:
SET STATISTICS TIME ON;
GO
ALTER TABLE SalesSource SWITCH TO SalesTarget;
GO
SET STATISTICS TIME OFF;

Commands completed successfully.

SQL Server parse and compile time: 
 CPU time = 15 ms, elapsed time = 15 ms.

SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 30 ms.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

Total execution time: 00:00:00.046

**4\. Verificar el número de renglones en cada tabla**

In [7]:
SELECT COUNT(*) FROM SalesSource; 
SELECT COUNT(*) FROM SalesTarget; 

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.155

(No column name)
0


(No column name)
1461000


## Carga de datos hacia una partición

**1\. Setup**

In [8]:
-- Drop objects if they already exist
IF EXISTS (SELECT * FROM sys.tables WHERE name = N'SalesSource')
  DROP TABLE SalesSource;
IF EXISTS (SELECT * FROM sys.tables WHERE name = N'SalesTarget')
  DROP TABLE SalesTarget;
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = N'psSales')
  DROP PARTITION SCHEME psSales;
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = N'pfSales')
  DROP PARTITION FUNCTION pfSales;

-- Create the Partition Function
CREATE PARTITION FUNCTION pfSales (DATE)
AS RANGE RIGHT FOR VALUES
('2013-01-01', '2014-01-01', '2015-01-01');
-- Create the Partition Scheme
CREATE PARTITION SCHEME psSales
AS PARTITION pfSales
ALL TO ([Primary]);
 
-- Create the Non-Partitioned Source Table (Heap) on the [PRIMARY] filegroup
CREATE TABLE SalesSource (
  SalesDate DATE,
  Quantity INT
) ON [PRIMARY];
-- Insert test data
INSERT INTO SalesSource(SalesDate, Quantity)
SELECT DATEADD(DAY,dates.n-1,'2012-01-01') AS SalesDate, qty.n AS Quantity
FROM dbo.GetNums(DATEDIFF(DD,'2012-01-01','2013-01-01')) dates
CROSS JOIN dbo.GetNums(1000) AS qty;
-- Create the Partitioned Target Table (Heap) on the Partition Scheme
CREATE TABLE SalesTarget (
  SalesDate DATE,
  Quantity INT
) ON psSales(SalesDate);
-- Insert test data
INSERT INTO SalesTarget(SalesDate, Quantity)
SELECT DATEADD(DAY,dates.n-1,'2013-01-01') AS SalesDate, qty.n AS Quantity
FROM dbo.GetNums(DATEDIFF(DD,'2013-01-01','2016-01-01')) dates
CROSS JOIN dbo.GetNums(1000) AS qty;

Partition scheme 'psSales' has been created successfully. 'PRIMARY' is marked as the next used filegroup in partition scheme 'psSales'.

(366000 rows affected)

(1095000 rows affected)

Total execution time: 00:00:04.308

**2\. Verificar número de renglones en las tablas**

In [9]:
SELECT COUNT(*) FROM SalesSource;

SELECT
  pstats.partition_number AS PartitionNumber
  ,pstats.row_count AS PartitionRowCount
FROM sys.dm_db_partition_stats AS pstats
WHERE pstats.object_id = OBJECT_ID('SalesTarget')
ORDER BY PartitionNumber;

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.098

(No column name)
366000


PartitionNumber,PartitionRowCount
1,0
2,365000
3,365000
4,365000


**3\. Realizar el intercambio de datos**

Si tratamos de usar el comando alter switch veremos que el intercambio falla debido a que la tabla de origen y la partición de destino no tienen las mismas constraints

In [10]:
SET STATISTICS TIME ON;
GO
ALTER TABLE SalesSource SWITCH TO SalesTarget PARTITION 1;
GO
SET STATISTICS TIME OFF;

Commands completed successfully.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 13 ms.

: Msg 4982, Level 16, State 1, Line 3
ALTER TABLE SWITCH statement failed. Check constraints of source table 'PartitionTestDB.dbo.SalesSource' allow values that are not allowed by range defined by partition 1 on target table 'PartitionTestDB.dbo.SalesTarget'.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

Total execution time: 00:00:00.019

Vamos a crear las constraints y posteriormente realizaremos el intercambio de datos

In [11]:
ALTER TABLE SalesSource
WITH CHECK ADD CONSTRAINT ckMinSalesDate
CHECK (SalesDate IS NOT NULL AND SalesDate >= '2012-01-01');
 
ALTER TABLE SalesSource
WITH CHECK ADD CONSTRAINT ckMaxSalesDate
CHECK (SalesDate IS NOT NULL AND SalesDate < '2013-01-01');

Commands completed successfully.

Total execution time: 00:00:00.159

In [11]:
SET STATISTICS TIME ON;
GO
ALTER TABLE SalesSource SWITCH TO SalesTarget PARTITION 1;
GO
SET STATISTICS TIME OFF;

Commands completed successfully.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 4 ms.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

Total execution time: 00:00:00.004

**4\. Verificar el número de renglones en cada tabla**

In [12]:
SELECT COUNT(*) FROM SalesSource;

SELECT
  pstats.partition_number AS PartitionNumber
  ,pstats.row_count AS PartitionRowCount
FROM sys.dm_db_partition_stats AS pstats
WHERE pstats.object_id = OBJECT_ID('SalesTarget')
ORDER BY PartitionNumber;

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.069

(No column name)
366000


PartitionNumber,PartitionRowCount
1,0
2,365000
3,365000
4,365000


## Mandar datos de partición hacia Archiving

**1\. Setup**

In [13]:
-- Drop objects if they already exist
IF EXISTS (SELECT * FROM sys.tables WHERE name = N'SalesSource')
  DROP TABLE SalesSource;
IF EXISTS (SELECT * FROM sys.tables WHERE name = N'SalesTarget')
  DROP TABLE SalesTarget;
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = N'psSales')
  DROP PARTITION SCHEME psSales;
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = N'pfSales')
  DROP PARTITION FUNCTION pfSales;
-- Create the Partition Function
CREATE PARTITION FUNCTION pfSales (DATE)
AS RANGE RIGHT FOR VALUES
('2013-01-01', '2014-01-01', '2015-01-01');
-- Create the Partition Scheme
CREATE PARTITION SCHEME psSales
AS PARTITION pfSales
ALL TO ([Primary]);
-- Create the Partitioned Source Table (Heap) on the Partition Scheme
CREATE TABLE SalesSource (
  SalesDate DATE,
  Quantity INT
) ON psSales(SalesDate);
-- Insert test data
INSERT INTO SalesSource(SalesDate, Quantity)
SELECT DATEADD(DAY,dates.n-1,'2012-01-01') AS SalesDate, qty.n AS Quantity
FROM GetNums(DATEDIFF(DD,'2012-01-01','2016-01-01')) dates
CROSS JOIN GetNums(1000) AS qty;
 
-- Create the Non-Partitioned Target Table (Heap) on the [PRIMARY] filegroup
CREATE TABLE SalesTarget (
  SalesDate DATE,
  Quantity INT
) ON [PRIMARY];

Partition scheme 'psSales' has been created successfully. 'PRIMARY' is marked as the next used filegroup in partition scheme 'psSales'.

(1461000 rows affected)

Total execution time: 00:00:03.792

**2\. Verificar número de renglones en las tablas**

In [16]:
SELECT
  pstats.partition_number AS PartitionNumber
  ,pstats.row_count AS PartitionRowCount
FROM sys.dm_db_partition_stats AS pstats
WHERE pstats.object_id = OBJECT_ID('SalesSource')
ORDER BY PartitionNumber;

SELECT COUNT(*) FROM SalesTarget;

(4 rows affected)

(1 row affected)

Total execution time: 00:00:00.010

PartitionNumber,PartitionRowCount
1,366000
2,365000
3,365000
4,365000


(No column name)
0


**3\. Realizar el intercambio de datos**

In [14]:
SET STATISTICS TIME ON;
GO
ALTER TABLE SalesSource SWITCH PARTITION 1 TO SalesTarget;
GO
SET STATISTICS TIME OFF;

Commands completed successfully.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

SQL Server Execution Times:
 CPU time = 0 ms, elapsed time = 1 ms.

SQL Server parse and compile time: 
 CPU time = 0 ms, elapsed time = 0 ms.

Total execution time: 00:00:00.001

**4\. Verificar el número de renglones**

In [13]:
SELECT
  pstats.partition_number AS PartitionNumber
  ,pstats.row_count AS PartitionRowCount
FROM sys.dm_db_partition_stats AS pstats
WHERE pstats.object_id = OBJECT_ID('SalesSource')
ORDER BY PartitionNumber;

SELECT COUNT(*) FROM SalesTarget;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.167

PartitionNumber,PartitionRowCount
1,366000


(No column name)
1095000
